In [14]:
!pip install requests ipywidgets

import requests
import re
import ipywidgets as widgets
from IPython.display import display, clear_output

In [13]:
pip install flask requests

In [15]:
# Xử lí nhiệm vụ
# Danh sách nhiệm vụ toàn cục
tasks = []

def add_task(task):
    """Thêm nhiệm vụ vào danh sách."""
    tasks.append(task)
    return f"Đã thêm nhiệm vụ: {task}"

def remove_task(index):
    """Xóa nhiệm vụ dựa theo chỉ số (1-indexed)."""
    try:
        removed = tasks.pop(index)
        return f"Đã xóa nhiệm vụ: {removed}"
    except IndexError:
        return "Chỉ số nhiệm vụ không hợp lệ."

def list_tasks():
    """Liệt kê toàn bộ nhiệm vụ hiện có."""
    if not tasks:
        return "Danh sách nhiệm vụ trống."
    output = "Danh sách nhiệm vụ:\n"
    for i, task in enumerate(tasks):
        output += f"{i+1}. {task}\n"
    return output


In [16]:
# Lấy tọa độ của thành phố sử dụng Geocoding API
def get_coordinates(city, state_code, country_code, limit, api_key):
    """
    Sử dụng API định vị của OpenWeatherMap để lấy tọa độ của thành phố.
    API: http://api.openweathermap.org/geo/1.0/direct?q={city name},{state code},{country code}&limit={limit}&appid={API key}
    """
    base_url = "http://api.openweathermap.org/geo/1.0/direct"
    params = {
        "q": f"{city},{state_code},{country_code}",
        "limit": limit,
        "appid": api_key
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data and len(data) > 0:
            # Lấy tọa độ của kết quả đầu tiên
            return data[0]["lat"], data[0]["lon"]
    return None, None

In [17]:
# Lấy thông tin thời tiết sử dụng tọa độ thu được

def get_weather(city, state_code, country_code, limit, api_key):
    """
    Sử dụng tọa độ lấy từ API định vị để gọi API thời tiết.
    """
    lat, lon = get_coordinates(city, state_code, country_code, limit, api_key)
    if lat is None or lon is None:
        return f"Không tìm thấy tọa độ của thành phố {city}. Vui lòng kiểm tra lại thông tin nhập vào."

    weather_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "lat": lat,
        "lon": lon,
        "appid": api_key,
        "units": "metric",   # Đơn vị đo: °C
        "lang": "vi"         # Ngôn ngữ: tiếng Việt
    }
    response = requests.get(weather_url, params=params)
    if response.status_code == 200:
        data = response.json()
        weather_desc = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        return f"Thời tiết ở {city} ({lat:.2f}, {lon:.2f}): {weather_desc} với nhiệt độ {temp}°C."
    else:
        return f"Không thể lấy dữ liệu thời tiết. Mã lỗi: {response.status_code}. Chi tiết: {response.text}"


In [20]:
# Xử lý lệnh người dùng
def process_command(command):
    """
    Xử lý lệnh của người dùng dựa trên từ khóa:
      - Nếu chứa "thời tiết": trả về thông tin thời tiết (sử dụng thông tin mặc định: Hanoi, VN)
      - Nếu chứa "thêm nhiệm vụ": thêm nhiệm vụ vào danh sách
      - Nếu chứa "xóa nhiệm vụ": xóa nhiệm vụ theo chỉ số
      - Nếu chứa "danh sách": liệt kê các nhiệm vụ
    """
    command_lower = command.lower()

    if "thời tiết" in command_lower:
        # Có thể mở rộng để trích xuất tên thành phố từ lệnh.
        # Ở ví dụ này, dùng mặc định: Hanoi, VN
        city = "Bienhoa"
        state = ""
        country = "VN"
        limit = 1
        # API key của bạn:
        api_key = "b17243e835dba3b7cac6e8c335b4d403"
        return get_weather(city, state, country, limit, api_key)

    elif "thêm nhiệm vụ" in command_lower:
        # Ví dụ: "Thêm nhiệm vụ: mua sắm"
        parts = command.split("nhiệm vụ:")
        if len(parts) > 1:
            task = parts[1].strip()
            return add_task(task)
        else:
            return "Lệnh không hợp lệ. Ví dụ: 'Thêm nhiệm vụ: mua sắm'"

    elif "xóa nhiệm vụ" in command_lower:
        # Ví dụ: "Xóa nhiệm vụ 1"
        match = re.search(r'\d+', command_lower)
        if match:
            index = int(match.group()) - 1  # chuyển từ số người dùng (1-indexed) sang index của list (0-indexed)
            return remove_task(index)
        else:
            return "Không tìm thấy chỉ số nhiệm vụ cần xóa."

    elif "danh sách" in command_lower:
        return list_tasks()

    else:
        return "Xin lỗi, tôi không hiểu lệnh của bạn. Vui lòng thử lại."

In [19]:
# Giao diện
# Tạo widget nhập liệu cho lệnh
command_text = widgets.Text(
    value="",
    placeholder=("Nhập lệnh của bạn, ví dụ:\n"
                 " - 'thời tiết hôm nay'\n"
                 " - 'thêm nhiệm vụ: mua sắm'\n"
                 " - 'xóa nhiệm vụ 1'\n"
                 " - 'danh sách'"),
    description="Lệnh:",
    layout=widgets.Layout(width='80%')
)

# Nút bấm thực hiện lệnh
button = widgets.Button(
    description="Thực hiện",
    button_style="success"
)

# Khu vực hiển thị kết quả
output_area = widgets.Output()

def on_button_click(b):
    with output_area:
        clear_output()  # Xóa kết quả cũ
        command = command_text.value.strip()
        result = process_command(command)
        print(result)
        command_text.value = ""  # Xóa ô nhập sau khi thực hiện lệnh

button.on_click(on_button_click)

# Hiển thị giao diện tương tác
display(command_text, button, output_area)

Text(value='', description='Lệnh:', layout=Layout(width='80%'), placeholder="Nhập lệnh của bạn, ví dụ:\n - 'th…

Button(button_style='success', description='Thực hiện', style=ButtonStyle())

Output()